In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

import cufflinks as cf
import numpy as np
import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline
import keyring
import requests
import json

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

from datetime import datetime
from pytz import timezone
from tzlocal import get_localzone

In [2]:
# defining credentials for auth
user = 'wramsey'
pwd = keyring.get_password("snow_prod", "wramsey")

# List of "sysparm_fields" being pulled
# u_repeat, upon_reject, sys_updated_on, number, u_vcc, state, sys_created_by, knowledge, u_external_customer_impacted, u_ach, cmdb_ci, impact, active, priority, sys_domain_path, business_duration, u_avidpay_check, u_core, u_ipay, u_avidpay_direct, short_description, work_start, u_check, notify, sys_class_name, closed_by, parent_incident, u_choice_6, reopened_by, u_utility, reassignment_count, assigned_to, u_board, escalation, upon_approval, u_payments_delayed, made_sla, u_money_movement, child_incidents, task_effective_number, resolved_by, sys_updated_by, opened_by, sys_created_on, sys_domain, u_dollar_impact, calendar_stc, closed_at, business_service, opened_at, caller_id, reopened_time, resolved_at, u_hub, subcategory, close_code, assignment_group, u_pir_requested, business_stc, calendar_duration, close_notes, sys_id, contact_type, incident_state, urgency, severity, approval, sys_mod_count, reopen_count, location, category

# Set the request parameters - assigned to support center in last 3 months
url = 'https://avidxchange.service-now.com/api/now/table/incident?sysparm_query=assignment_group%3Dfe169ea0db5e2700bd3ec170ba961941%5Eopened_at%3Ejavascript%3Ags.beginningOfLast3Months()&sysparm_fields=u_repeat%2C%20upon_reject%2C%20sys_updated_on%2C%20number%2C%20u_vcc%2C%20state%2C%20sys_created_by%2C%20knowledge%2C%20u_external_customer_impacted%2C%20u_ach%2C%20cmdb_ci%2C%20impact%2C%20active%2C%20priority%2C%20sys_domain_path%2C%20business_duration%2C%20u_avidpay_check%2C%20u_core%2C%20u_ipay%2C%20u_avidpay_direct%2C%20short_description%2C%20work_start%2C%20u_check%2C%20notify%2C%20sys_class_name%2C%20closed_by%2C%20parent_incident%2C%20u_choice_6%2C%20reopened_by%2C%20u_utility%2C%20reassignment_count%2C%20assigned_to%2C%20u_board%2C%20escalation%2C%20upon_approval%2C%20u_payments_delayed%2C%20made_sla%2C%20u_money_movement%2C%20child_incidents%2C%20task_effective_number%2C%20resolved_by%2C%20sys_updated_by%2C%20opened_by%2C%20sys_created_on%2C%20sys_domain%2C%20u_dollar_impact%2C%20calendar_stc%2C%20closed_at%2C%20business_service%2C%20opened_at%2C%20caller_id%2C%20reopened_time%2C%20resolved_at%2C%20u_hub%2C%20subcategory%2C%20close_code%2C%20assignment_group%2C%20u_pir_requested%2C%20business_stc%2C%20calendar_duration%2C%20close_notes%2C%20sys_id%2C%20contact_type%2C%20incident_state%2C%20urgency%2C%20severity%2C%20approval%2C%20sys_mod_count%2C%20reopen_count%2C%20location%2C%20category&sysparm_limit=3000'

# Set proper headers
headers = {"Content-Type":"application/json","Accept":"application/json"}

# Do the HTTP request
response = requests.get(url, auth=(user, pwd), headers=headers )

# Check for HTTP codes other than 200
if response.status_code != 200:
    print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
    print('Status:', response.status_code, '\n')
    print('Headers:', response.headers, '\n')
    data = response.json()
    print(json.dumps(data, indent=2))

# Decode the JSON response into a dictionary and use the data
data = response.json()

# this turns the dictionary into a pandas df
df_main = pd.DataFrame.from_dict(data['result'])
pd.set_option('max_columns', None)

### Read data and transform datatypes to datetime and adding additional time columns

In [3]:
# read in data, expand df to show all columns
# this .csv is a file containing the last 90 days worth of activities
# df_main = pd.read_csv(r"C:\tiffany.csv")
# pd.set_option('max_columns', None)

# Change datatypes of all time-based columns
# time_columns = ['sys_updated_on', 'sys_created_on', 'closed_at', 'opened_at', 'business_duration', 'reopened_time',
#                 'resolved_at', 'work_start', 'calendar_duration']

# for item in time_columns:
#     df_main[item] = pd.to_datetime(df_main[item])

# Create additional time columns
nyc = timezone('America/New_York') 
df_main['resolved_at'] = pd.to_datetime(df_main['resolved_at']).dt.tz_localize('UTC').dt.tz_convert(nyc)
df_main['work_start'] = pd.to_datetime(df_main['work_start']).dt.tz_localize('UTC').dt.tz_convert(nyc)
df_main['calendar_duration'] = pd.to_datetime(df_main['calendar_duration']).dt.tz_localize('UTC').dt.tz_convert(nyc)
df_main['reopened_time'] = pd.to_datetime(df_main['reopened_time']).dt.tz_localize('UTC').dt.tz_convert(nyc)
df_main['opened_at'] = pd.to_datetime(df_main['opened_at']).dt.tz_localize('UTC').dt.tz_convert(nyc)
df_main['closed_at'] = pd.to_datetime(df_main['closed_at']).dt.tz_localize('UTC').dt.tz_convert(nyc)
df_main['sys_updated_on'] = pd.to_datetime(df_main['sys_updated_on']).dt.tz_localize('UTC').dt.tz_convert(nyc)
df_main['sys_created_on'] = pd.to_datetime(df_main['sys_created_on']).dt.tz_localize('UTC').dt.tz_convert(nyc)
df_main['business_duration'] = pd.to_datetime(df_main['business_duration']).dt.tz_localize('UTC').dt.tz_convert(nyc)

df_main['opened_at_hour'] = df_main['opened_at'].dt.hour
df_main['opened_at_month'] = df_main['opened_at'].dt.month
df_main['opened_at_day'] = df_main['opened_at'].dt.day
df_main['opened_at_day_of_year'] = df_main['opened_at'].dt.dayofyear
df_main['opened_at_clone'] = df_main['opened_at']
df_main['resolution_hour'] = df_main['resolved_at'].dt.hour
df_main['resolution_time_length'] = df_main.resolved_at - df_main.opened_at
df_main['resolution_time_total_hours'] = df_main['resolution_time_length'].dt.total_seconds()/3600
# print(type(df_main['opened_at_clone'].iloc[0]))
df_main[['opened_at', 'task_effective_number', 'opened_at_hour', 'opened_at_month', 'opened_at_day', 'opened_at_day_of_year',
         'opened_at_clone', 'resolution_time_length',
         'resolution_time_total_hours']].head(5).sort_values('opened_at_day').head(2)

# place all transformations here
df_main = df_main.replace(['2d5d343adb2d17009f9aec51ca9619e9'],['five9'])
df_main = df_main.replace(['55da5d83db3057007f3dec51ca96190c'],['Unknown_CI'])

# Get today's date
sixty_days = datetime.today() - timedelta(60)

# to only include results after today's data
df_main = df_main[ df_main['opened_at'] > sixty_days.date().strftime('%Y-%m-%d') ]

# where contact type not 'event'
df_main = df_main[ df_main['contact_type'] != 'Event' ]

### What is the timeframe of this data

In [4]:
# to pull days as int (for calculations)
# print((df_main['opened_at'].max() - df_main['opened_at'].min()).days)
print('Time range of data:       '+str(df_main['opened_at'].max() - df_main['opened_at'].min()))
print('earliest incident:        '+str(df_main['opened_at'].dt.date.min()))
print('latest incident:          '+str(df_main['opened_at'].dt.date.max()))
print('Number of incidents within this period: ' + str(df_main['opened_at'].count()))
print('Number of incidents AFTER 6PM: ' + str(df_main[ df_main['opened_at_hour'] >= 18]['opened_at'].count()))
print('\033[1m' + 'Percentage of tickets received after 6pm: ' + '\033[0m' + str(round((df_main[ df_main['opened_at_hour'] >= 18]['opened_at'].count())
                                                               / (df_main['opened_at'].count()), 2)) + ' percent')

Time range of data:       58 days 12:41:14
earliest incident:        2021-04-05
latest incident:          2021-06-02
Number of incidents within this period: 1384
Number of incidents AFTER 6PM: 32
Percentage of tickets received after 6pm: 0.02 percent


In [5]:
# Incidents included in this report
df_time = df_main[ df_main['opened_at_hour'] >= 18]
df_time[['opened_at', 'task_effective_number', 'sys_created_by', 'contact_type']].sort_values('opened_at').head()

,opened_at,task_effective_number,sys_created_by,contact_type
470,2021-04-05 22:37:54-04:00,INC0068805,SLakamsani,Self Service
1835,2021-04-06 18:05:13-04:00,INC0068913,SLakamsani,Self Service
1522,2021-04-08 19:34:22-04:00,INC0069108,Mbowens,Self Service
1031,2021-04-13 18:12:17-04:00,INC0069444,SDelgado,Self Service
840,2021-04-14 18:02:17-04:00,INC0069539,X103427,Self Service


### How many INC are opened per hour - Total Day Overview

In [6]:
df_inc_per_hr = df_main['task_effective_number'].groupby(df_main['opened_at'].dt.hour).count()
# df_inc_per_hr

fig = px.bar(df_inc_per_hr, x=df_inc_per_hr.index, y='task_effective_number',
             labels={'opened_at':'Hour Opened - 24hr Clock', 'task_effective_number':'Total Count of Incidents'},
            title='INC Opened Per Hour (Past 60 Days) - Vertical Line at 6pm')
fig.add_vline(x=18.5, line_dash="dash", line_color="green")
fig.show()

### Total count of INC opened per day - After 6pm

In [7]:
# Set to after 6pm only
df_main = df_main[ df_main['opened_at_hour'] >= 18]

df_daily = df_main.copy(deep=True)
df_daily.set_index('opened_at_clone', inplace=True)
df_daily_resample_b = df_daily['task_effective_number'].resample('B').count()

fig = px.line(df_daily_resample_b, x=df_daily_resample_b.index, y=df_daily['task_effective_number'].resample('B').count(),
              title='Count of Total INC Opened Per Day After 6pm (Weekends not included)')
fig.show()

### Who opens the most tickets after 6pm? (by count)

In [8]:
# show number of tickets by INC-caller 
fig = px.bar(df_main['sys_created_by'].value_counts(), x=df_main['sys_created_by'].value_counts().index,
             y='sys_created_by', labels={'x':'Username', 'sys_created_by':'Number of INC opened'},
            title='Who opens the most tickets after 6pm? (by count)')
fig.update_xaxes(range=[0,50])
fig.show()

### Types of INC received after 6 pm (by percentage)

In [9]:
# df_lots_of_time['subcategory'].value_counts().head()
df_cat = df_main[ df_main['subcategory'] != '' ]
fig = px.bar(df_cat['subcategory'][df_cat['subcategory']!=0].value_counts(normalize=True, ascending=False),
             x=df_cat['subcategory'][df_cat['subcategory']!=0].value_counts().index,
             y='subcategory', title='Types of INC received after 6 pm (by percentage)')
fig.show()